In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size
img_size = (469, 369)

# Load CSV files
df_train = pd.read_csv(
    "/home/arush/GW_Project_1/Data_Generation/Continous_Check/cont_data_train.csv"
)
df_test = pd.read_csv(
    "/home/arush/GW_Project_1/Data_Generation/Continous_Check/cont_data_test.csv"
)
df_val = pd.read_csv(
    "/home/arush/GW_Project_1/Data_Generation/Continous_Check/cont_data_val.csv"
)

# Data generator for scaling images
scale = ImageDataGenerator(rescale=1.0 / 255)

# Create train, test, and validation generators
train_generator = scale.flow_from_dataframe(
    df_train,
    x_col="Path",
    y_col="Label",
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
)

test_generator = scale.flow_from_dataframe(
    df_test,
    x_col="Path",
    y_col="Label",
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
)

val_generator = scale.flow_from_dataframe(
    df_val,
    x_col="Path",
    y_col="Label",
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
)

Found 7071 validated image filenames.
Found 884 validated image filenames.
Found 884 validated image filenames.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the regression model
model = Sequential()
model.add(
    Conv2D(32, (3, 3), activation="relu", input_shape=(img_size[0], img_size[1], 3))
)
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(
    Dense(1, activation="linear")
)  # Single neuron with linear activation for regression

# Compile the model
model.compile(
    optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error"]
)

# Print model summary
model.summary()

/home/arush/anaconda3/envs/python39_1/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 467, 367, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 233, 183, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 231, 181, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 115, 90, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 113, 88, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 56, 44, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 54, 42, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 27, 21, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 72576)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     4,644,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,885,825 (18.64 MB)

 Trainable params: 4,885,825 (18.64 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define callbacks
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.001)

# Fit the model with callbacks
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
)

Epoch 1/20
221/221 ━━━━━━━━━━━━━━━━━━━━ 814s 4s/step - loss: 66.3889 - mean_absolute_error: 6.4450 - val_loss: 55.5778 - val_mean_absolute_error: 5.8377 - learning_rate: 0.0010
Epoch 2/20
221/221 ━━━━━━━━━━━━━━━━━━━━ 758s 3s/step - loss: 53.0584 - mean_absolute_error: 5.7902 - val_loss: 55.1343 - val_mean_absolute_error: 5.8478 - learning_rate: 0.0010
Epoch 3/20
221/221 ━━━━━━━━━━━━━━━━━━━━ 768s 3s/step - loss: 53.2325 - mean_absolute_error: 5.7984 - val_loss: 55.0349 - val_mean_absolute_error: 5.8644 - learning_rate: 0.0010
Epoch 4/20
221/221 ━━━━━━━━━━━━━━━━━━━━ 758s 3s/step - loss: 52.5704 - mean_absolute_error: 5.7492 - val_loss: 56.2054 - val_mean_absolute_error: 5.8512 - learning_rate: 0.0010
Epoch 5/20
 71/221 ━━━━━━━━━━━━━━━━━━━━ 8:34 3s/step - loss: 49.3908 - mean_absolute_error: 5.6068

KeyboardInterrupt: 

In [5]:
print(history.history.keys())

dict_keys([])
